In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=500):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            #If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        plt.savefig('mnist_{:04d}.png'.format(epoch))
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 64)         

64 [D loss: 0.765931, acc.: 60.94%] [G loss: 0.986627]
65 [D loss: 0.721232, acc.: 60.94%] [G loss: 1.275251]
66 [D loss: 0.680832, acc.: 62.50%] [G loss: 1.406695]
67 [D loss: 0.609917, acc.: 73.44%] [G loss: 1.088296]
68 [D loss: 0.981764, acc.: 42.19%] [G loss: 1.073561]
69 [D loss: 0.782649, acc.: 65.62%] [G loss: 0.897904]
70 [D loss: 0.696804, acc.: 59.38%] [G loss: 1.132872]
71 [D loss: 0.726988, acc.: 62.50%] [G loss: 1.038459]
72 [D loss: 0.773425, acc.: 60.94%] [G loss: 1.077285]
73 [D loss: 0.770572, acc.: 56.25%] [G loss: 1.276006]
74 [D loss: 0.886087, acc.: 53.12%] [G loss: 1.062726]
75 [D loss: 0.813996, acc.: 50.00%] [G loss: 0.962229]
76 [D loss: 0.648405, acc.: 68.75%] [G loss: 0.940108]
77 [D loss: 0.837181, acc.: 54.69%] [G loss: 1.461599]
78 [D loss: 0.778248, acc.: 54.69%] [G loss: 1.143674]
79 [D loss: 0.929257, acc.: 48.44%] [G loss: 1.150515]
80 [D loss: 0.706795, acc.: 57.81%] [G loss: 1.400337]
81 [D loss: 0.724308, acc.: 59.38%] [G loss: 1.125364]
82 [D loss

211 [D loss: 0.445803, acc.: 79.69%] [G loss: 0.994446]
212 [D loss: 0.434584, acc.: 81.25%] [G loss: 1.070702]
213 [D loss: 0.498770, acc.: 71.88%] [G loss: 1.357783]
214 [D loss: 0.687405, acc.: 64.06%] [G loss: 1.087483]
215 [D loss: 0.858841, acc.: 51.56%] [G loss: 1.228106]
216 [D loss: 0.567638, acc.: 70.31%] [G loss: 1.558797]
217 [D loss: 0.601730, acc.: 67.19%] [G loss: 1.060551]
218 [D loss: 0.654168, acc.: 65.62%] [G loss: 1.081659]
219 [D loss: 0.514160, acc.: 76.56%] [G loss: 1.340601]
220 [D loss: 0.581432, acc.: 64.06%] [G loss: 1.193119]
221 [D loss: 0.816818, acc.: 51.56%] [G loss: 0.921931]
222 [D loss: 0.571692, acc.: 71.88%] [G loss: 1.036916]
223 [D loss: 0.368097, acc.: 85.94%] [G loss: 1.370769]
224 [D loss: 0.568311, acc.: 70.31%] [G loss: 1.478949]
225 [D loss: 0.762633, acc.: 56.25%] [G loss: 1.328505]
226 [D loss: 0.468586, acc.: 75.00%] [G loss: 1.193520]
227 [D loss: 0.792515, acc.: 51.56%] [G loss: 1.367430]
228 [D loss: 1.039052, acc.: 35.94%] [G loss: 1.

358 [D loss: 0.317701, acc.: 89.06%] [G loss: 0.734950]
359 [D loss: 0.466192, acc.: 81.25%] [G loss: 1.303423]
360 [D loss: 0.863321, acc.: 54.69%] [G loss: 0.873279]
361 [D loss: 1.036184, acc.: 53.12%] [G loss: 0.790456]
362 [D loss: 0.148382, acc.: 93.75%] [G loss: 1.111515]
363 [D loss: 0.939352, acc.: 59.38%] [G loss: 0.583216]
364 [D loss: 0.371163, acc.: 78.12%] [G loss: 0.480333]
365 [D loss: 0.221532, acc.: 92.19%] [G loss: 0.759287]
366 [D loss: 0.105925, acc.: 98.44%] [G loss: 1.059742]
367 [D loss: 0.370718, acc.: 84.38%] [G loss: 0.780032]
368 [D loss: 0.198593, acc.: 93.75%] [G loss: 0.722568]
369 [D loss: 0.157173, acc.: 96.88%] [G loss: 0.880823]
370 [D loss: 0.068750, acc.: 100.00%] [G loss: 1.125017]
371 [D loss: 0.052075, acc.: 100.00%] [G loss: 1.278492]
372 [D loss: 0.290448, acc.: 85.94%] [G loss: 1.161530]
373 [D loss: 0.078239, acc.: 98.44%] [G loss: 0.796791]
374 [D loss: 0.149374, acc.: 93.75%] [G loss: 0.949056]
375 [D loss: 1.256237, acc.: 37.50%] [G loss: 

504 [D loss: 0.675293, acc.: 59.38%] [G loss: 1.235947]
505 [D loss: 0.467038, acc.: 79.69%] [G loss: 0.926112]
506 [D loss: 0.126259, acc.: 96.88%] [G loss: 1.124778]
507 [D loss: 0.082637, acc.: 100.00%] [G loss: 1.222930]
508 [D loss: 0.100683, acc.: 100.00%] [G loss: 1.170320]
509 [D loss: 0.580762, acc.: 68.75%] [G loss: 0.875234]
510 [D loss: 0.252160, acc.: 92.19%] [G loss: 0.751497]
511 [D loss: 2.512761, acc.: 6.25%] [G loss: 0.641549]
512 [D loss: 0.128297, acc.: 96.88%] [G loss: 0.644895]
513 [D loss: 0.023270, acc.: 100.00%] [G loss: 0.940162]
514 [D loss: 0.056419, acc.: 100.00%] [G loss: 0.974780]
515 [D loss: 0.051521, acc.: 100.00%] [G loss: 1.354460]
516 [D loss: 0.049502, acc.: 100.00%] [G loss: 1.349805]
517 [D loss: 0.165074, acc.: 95.31%] [G loss: 1.789297]
518 [D loss: 0.004972, acc.: 100.00%] [G loss: 2.113282]
519 [D loss: 0.010128, acc.: 100.00%] [G loss: 1.471022]
520 [D loss: 0.005744, acc.: 100.00%] [G loss: 1.880738]
521 [D loss: 0.003358, acc.: 100.00%] [G

...

3352 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.087515]
3353 [D loss: 0.000043, acc.: 100.00%] [G loss: 0.155381]
3354 [D loss: 0.000012, acc.: 100.00%] [G loss: 0.193391]
3355 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.130770]
3356 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.225554]
3357 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.168186]
3358 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.120821]
3359 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.134991]
3360 [D loss: 0.000014, acc.: 100.00%] [G loss: 0.180843]
3361 [D loss: 0.000012, acc.: 100.00%] [G loss: 0.221951]
3362 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.157892]
3363 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.214451]
3364 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.170397]
3365 [D loss: 0.000015, acc.: 100.00%] [G loss: 0.203679]
3366 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.108081]
3367 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.098940]
3368 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.093876]
3369 [D loss: 

3494 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.099191]
3495 [D loss: 0.000010, acc.: 100.00%] [G loss: 0.091636]
3496 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.146618]
3497 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.109472]
3498 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.058026]
3499 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.097240]
3500 [D loss: 0.000019, acc.: 100.00%] [G loss: 0.151286]
3501 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.085683]
3502 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.085814]
3503 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.129884]
3504 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.093180]
3505 [D loss: 0.000010, acc.: 100.00%] [G loss: 0.136338]
3506 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.157385]
3507 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.146560]
3508 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.177022]
3509 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.148316]
3510 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.157753]
3511 [D loss: 

3636 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.227181]
3637 [D loss: 0.000015, acc.: 100.00%] [G loss: 0.340647]
3638 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.192881]
3639 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.223154]
3640 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.209117]
3641 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.127579]
3642 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.198983]
3643 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.233389]
3644 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.084603]
3645 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.143283]
3646 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.102741]
3647 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.152583]
3648 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.132169]
3649 [D loss: 0.000015, acc.: 100.00%] [G loss: 0.133919]
3650 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.131723]
3651 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.115205]
3652 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.094376]
3653 [D loss: 

3778 [D loss: 0.000017, acc.: 100.00%] [G loss: 0.317808]
3779 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.179569]
3780 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.132739]
3781 [D loss: 0.000009, acc.: 100.00%] [G loss: 0.185501]
3782 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.148832]
3783 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.120419]
3784 [D loss: 0.000009, acc.: 100.00%] [G loss: 0.114775]
3785 [D loss: 0.000009, acc.: 100.00%] [G loss: 0.152656]
3786 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.169317]
3787 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.096246]
3788 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.103109]
3789 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.180580]
3790 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.085193]
3791 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.078842]
3792 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.114358]
3793 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.086587]
3794 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.093322]
3795 [D loss: 

3920 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.163257]
3921 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.149818]
3922 [D loss: 0.000007, acc.: 100.00%] [G loss: 0.196918]
3923 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.121300]
3924 [D loss: 0.000031, acc.: 100.00%] [G loss: 0.362915]
3925 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.199222]
3926 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.197355]
3927 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.128218]
3928 [D loss: 0.000016, acc.: 100.00%] [G loss: 0.345653]
3929 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.221778]
3930 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.235419]
3931 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.192994]
3932 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.282456]
3933 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.182971]
3934 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.172706]
3935 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.154177]
3936 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.206383]
3937 [D loss: 